# Classifiers evaluation and comparison
Nikiforova Kristina

### Import libs

In [24]:
from scipy.io import arff
import pandas as pd
import numpy as np
import itertools
from sklearn import *
%pylab inline
warnings.filterwarnings('ignore')

Populating the interactive namespace from numpy and matplotlib


### Load dataset

#### 1. digits

In [25]:
digits = datasets.load_digits()
df = pd.DataFrame(digits.data)
df['target'] = digits.target
df.head()

,0,1,2,3,4,5,6,7,8,9,...,55,56,57,58,59,60,61,62,63,target
0,0.0,0.0,5.0,13.0,9.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,6.0,13.0,10.0,0.0,0.0,0.0,0
1,0.0,0.0,0.0,12.0,13.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,11.0,16.0,10.0,0.0,0.0,1
2,0.0,0.0,0.0,4.0,15.0,12.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,11.0,16.0,9.0,0.0,2
3,0.0,0.0,7.0,15.0,13.0,1.0,0.0,0.0,0.0,8.0,...,0.0,0.0,0.0,7.0,13.0,13.0,9.0,0.0,0.0,3
4,0.0,0.0,0.0,1.0,11.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,2.0,16.0,4.0,0.0,0.0,4


#### 2. breast_cancer

In [26]:
breast_canсer = datasets.load_breast_cancer()
bcf = pd.DataFrame(breast_canсer.data)
bcf['target'] = breast_canсer.target
bcf.head()

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0


#### 3. wine

In [27]:
wine = datasets.load_wine()
wf = pd.DataFrame(wine.data)
wf['target'] = wine.target
wf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,0


### сonversion

In [28]:
target_d =  df['target'] 
data_d = df.drop(columns = ['target'])

target_bc = bcf['target'] 
data_bc = bcf.drop(columns = ['target'])

target_w = wf['target'] 
data_w = wf.drop(columns = ['target'])

print ('Digits data size: {} instances \nBreast_cancer data size: {} instances\nWine data size:'
       ' {} instances'.format(len(data_d), len(data_bc),len(data_w)))

Digits data size: 1797 instances 
Breast_cancer data size: 569 instances
Wine data size: 178 instances


### Function for classifier settings

In [29]:
def load_classifier(classifier, params, train_x, test_x, train_y, folds, scor): 
    grid = model_selection.GridSearchCV(classifier, params, refit=True, scoring=f1_scorer, iid = True,
                                     cv=model_selection.StratifiedKFold(n_splits=folds))
    grid_best = grid.fit(train_x, train_y)
    print("Best hyper-parameters: {}".format(grid_best.best_params_))
    predictions = grid_best.predict(test_x)
    #metrics.ploat_confusion_matrix(grid_best, test_data, test_labe)
    f1 = metrics.f1_score(test_y, predictions, average='weighted')
    print ("F1 = {:.3f}".format(f1))
    #print(metrics.classification_report(test_y, predictions))

### cross-validation predictions

In [30]:
def cross(model, test_x, test_y):
    predictions = model_selection.cross_val_predict(model, test_x, test_y)
    f1 = metrics.f1_score(test_y, predictions, average='weighted')
    print('Cross-validation predictions:')
    print ("f1 = {:.3f}".format(f1))
    print(metrics.classification_report(test_y, predictions))

## Digits

In [41]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data_d, target_d, test_size = 0.2)
folds = 5
f1_scorer = metrics.make_scorer(metrics.f1_score, average='weighted')

### Baseline

In [40]:
model = dummy.DummyClassifier(strategy='most_frequent')
model.fit (data_d, target_d)
predictions = model.predict(data_d)
accuracy_base = metrics.accuracy_score(target_d, predictions)
print ("Accuracy = {:.3f}".format(accuracy_base))
print(metrics.classification_report(target_d, predictions))

#plot_confusion_matrix(model, data_d, target_d)

Accuracy = 0.102
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       178
           1       0.00      0.00      0.00       182
           2       0.00      0.00      0.00       177
           3       0.10      1.00      0.18       183
           4       0.00      0.00      0.00       181
           5       0.00      0.00      0.00       182
           6       0.00      0.00      0.00       181
           7       0.00      0.00      0.00       179
           8       0.00      0.00      0.00       174
           9       0.00      0.00      0.00       180

    accuracy                           0.10      1797
   macro avg       0.01      0.10      0.02      1797
weighted avg       0.01      0.10      0.02      1797



### Find best classifier

### LogisticRegression


In [12]:
logistic_param = {'penalty': ['l1', 'l2'],'C': [1, 10, 100], 'max_iter': [100, 300],'tol': [1e-3, 1e-4]}
logistic_classifier = linear_model.LogisticRegression(random_state=1, solver='liblinear')
load_classifier(logistic_classifier,logistic_param,train_x, test_x, train_y, folds, f1_scorer)
cross(logistic_classifier, test_x,test_y)

Best hyper-parameters: {'C': 1, 'max_iter': 100, 'penalty': 'l1', 'tol': 0.001}
F1 = 0.969
Cross-validation predictions:
f1 = 0.950
              precision    recall  f1-score   support

           0       0.93      1.00      0.97        43
           1       0.86      0.95      0.90        39
           2       1.00      0.97      0.98        32
           3       0.95      0.95      0.95        40
           4       1.00      0.98      0.99        46
           5       0.96      0.90      0.93        29
           6       1.00      1.00      1.00        35
           7       1.00      1.00      1.00        27
           8       0.85      0.85      0.85        33
           9       0.97      0.89      0.93        36

    accuracy                           0.95       360
   macro avg       0.95      0.95      0.95       360
weighted avg       0.95      0.95      0.95       360



### SVM

In [11]:
svc_param = {'kernel': ['linear','rbf'], 'gamma': [1e-4, 'scale'], 'C': [1, 10]}
svc_classifier = svm.SVC(probability=True)
load_classifier(svc_classifier, svc_param, train_x, test_x, train_y, folds, f1_scorer)
cross(svc_classifier, test_x,test_y)

Best hyper-parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'rbf'}
F1 = 0.989
Cross-validation predictions:
f1 = 0.096
              precision    recall  f1-score   support

           0       1.00      0.19      0.31        43
           1       1.00      0.15      0.27        39
           2       0.00      0.00      0.00        32
           3       0.00      0.00      0.00        40
           4       0.13      1.00      0.23        46
           5       0.00      0.00      0.00        29
           6       0.00      0.00      0.00        35
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        33
           9       0.00      0.00      0.00        36

    accuracy                           0.17       360
   macro avg       0.21      0.13      0.08       360
weighted avg       0.24      0.17      0.10       360



### DecisionTreeClassifier

In [13]:
decision_param = {'presort':[True, False], 'max_depth' : [ 2, 3, 5, 7, 10], 'max_features' : ['auto', 1, 2, 3, None],
                  'min_samples_leaf' : [1, 2, 3, 4, 5]}
decision_classifier = tree.DecisionTreeClassifier(random_state=1)
load_classifier(decision_classifier, decision_param ,train_x, test_x, train_y,folds, f1_scorer)
cross(decision_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': 10, 'max_features': None, 'min_samples_leaf': 1, 'presort': True}
F1 = 0.863
Cross-validation predictions:
f1 = 0.718
              precision    recall  f1-score   support

           0       0.87      0.93      0.90        43
           1       0.64      0.74      0.69        39
           2       0.79      0.72      0.75        32
           3       0.68      0.70      0.69        40
           4       0.83      0.74      0.78        46
           5       0.63      0.59      0.61        29
           6       0.88      0.80      0.84        35
           7       0.73      0.81      0.77        27
           8       0.41      0.45      0.43        33
           9       0.69      0.61      0.65        36

    accuracy                           0.72       360
   macro avg       0.72      0.71      0.71       360
weighted avg       0.72      0.72      0.72       360



### NaiveBayes

In [15]:
naive_param = {'var_smoothing' : [ 1e-3, 1e-5, 1e-7, 1e-9]}
naive_classifier = naive_bayes.GaussianNB()
load_classifier(naive_classifier, naive_param,train_x, test_x, train_y, folds, f1_scorer)
cross(naive_classifier, test_x,test_y)

Best hyper-parameters: {'var_smoothing': 0.001}
F1 = 0.895
Cross-validation predictions:
f1 = 0.815
              precision    recall  f1-score   support

           0       1.00      0.93      0.96        43
           1       0.72      0.72      0.72        39
           2       0.94      0.94      0.94        32
           3       0.75      0.68      0.71        40
           4       0.93      0.85      0.89        46
           5       0.77      0.93      0.84        29
           6       0.94      0.86      0.90        35
           7       0.77      0.89      0.83        27
           8       0.60      0.79      0.68        33
           9       0.73      0.61      0.67        36

    accuracy                           0.81       360
   macro avg       0.82      0.82      0.81       360
weighted avg       0.82      0.81      0.81       360



### RandomForestCLassifier

In [16]:
forest_param = {'max_depth' : [None, 8, 10, 13], 'n_estimators' : [10, 50, 100, 250], 'max_features' : ['auto', 'log2']}
forest_classifier = ensemble.RandomForestClassifier(random_state=1)
load_classifier(forest_classifier, forest_param, train_x, test_x, train_y,folds, f1_scorer)
cross(forest_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': None, 'max_features': 'log2', 'n_estimators': 250}
F1 = 0.980
Cross-validation predictions:
f1 = 0.859
              precision    recall  f1-score   support

           0       0.91      0.91      0.91        43
           1       0.90      0.92      0.91        39
           2       0.86      0.94      0.90        32
           3       0.79      0.78      0.78        40
           4       0.88      0.93      0.91        46
           5       0.88      0.76      0.81        29
           6       0.92      0.94      0.93        35
           7       0.87      0.96      0.91        27
           8       0.71      0.67      0.69        33
           9       0.88      0.78      0.82        36

    accuracy                           0.86       360
   macro avg       0.86      0.86      0.86       360
weighted avg       0.86      0.86      0.86       360



### AdaBoostClassifier

In [17]:
boost_param = {'n_estimators' : [50, 100], 'learning_rate' : [1, 0.7], 'algorithm': ['SAMME', 'SAMME.R']}
boost_classifier = ensemble.AdaBoostClassifier()
load_classifier(boost_classifier, boost_param, train_x, test_x, train_y,folds, f1_scorer)
cross(boost_classifier, test_x,test_y)

Best hyper-parameters: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 100}
F1 = 0.803
Cross-validation predictions:
f1 = 0.229
              precision    recall  f1-score   support

           0       0.67      0.51      0.58        43
           1       0.00      0.00      0.00        39
           2       0.13      0.31      0.18        32
           3       0.19      0.38      0.25        40
           4       0.23      0.48      0.31        46
           5       0.80      0.14      0.24        29
           6       0.38      0.37      0.38        35
           7       0.00      0.00      0.00        27
           8       0.00      0.00      0.00        33
           9       0.23      0.19      0.21        36

    accuracy                           0.26       360
   macro avg       0.26      0.24      0.21       360
weighted avg       0.26      0.26      0.23       360



### VotingCLassifier

In [18]:
voting_param = {'voting' : ['hard', 'soft'], 'flatten_transform' :[True, False, None]}
voting_classifier = ensemble.VotingClassifier(estimators=[('lr', logistic_classifier), 
                                            ('rf', ensemble.RandomForestClassifier(n_estimators=50, random_state=1)), 
                                            ('gnb', naive_classifier)])
load_classifier(voting_classifier, voting_param, train_x, test_x, train_y, folds, f1_scorer)
cross(voting_classifier, test_x,test_y)

Best hyper-parameters: {'flatten_transform': True, 'voting': 'hard'}
F1 = 0.964
Cross-validation predictions:
f1 = 0.931
              precision    recall  f1-score   support

           0       0.96      1.00      0.98        43
           1       0.88      0.95      0.91        39
           2       1.00      0.97      0.98        32
           3       0.88      0.88      0.88        40
           4       0.98      0.93      0.96        46
           5       1.00      0.90      0.95        29
           6       1.00      1.00      1.00        35
           7       0.90      0.96      0.93        27
           8       0.83      0.88      0.85        33
           9       0.91      0.83      0.87        36

    accuracy                           0.93       360
   macro avg       0.93      0.93      0.93       360
weighted avg       0.93      0.93      0.93       360



## breast_cancer

### Baseline

In [20]:
model = dummy.DummyClassifier(strategy='most_frequent')
model.fit (data_bc, target_bc)
predictions = model.predict(data_bc)
accuracy_base = metrics.accuracy_score(target_bc, predictions)
print ("Accuracy = {:.3f}".format(accuracy_base))
print(metrics.classification_report(target_bc, predictions))
#metrics.plot_confusion_matrix(model, data_d, target_d)

Accuracy = 0.627
              precision    recall  f1-score   support

           0       0.00      0.00      0.00       212
           1       0.63      1.00      0.77       357

    accuracy                           0.63       569
   macro avg       0.31      0.50      0.39       569
weighted avg       0.39      0.63      0.48       569



### Find best classifier

In [21]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data_bc, target_bc, test_size = 0.2)
folds = 5
f1_scorer  = metrics.make_scorer(metrics.f1_score, average='weighted')

### Logistic Regression

In [22]:
load_classifier(logistic_classifier,logistic_param,train_x, test_x, train_y, folds, f1_scorer)
cross(logistic_classifier, test_x,test_y)

Best hyper-parameters: {'C': 100, 'max_iter': 100, 'penalty': 'l1', 'tol': 0.001}
F1 = 0.938
Cross-validation predictions:
f1 = 0.912
              precision    recall  f1-score   support

           0       0.90      0.86      0.88        44
           1       0.92      0.94      0.93        70

    accuracy                           0.91       114
   macro avg       0.91      0.90      0.91       114
weighted avg       0.91      0.91      0.91       114



### SVM

In [23]:
load_classifier(svc_classifier, svc_param, train_x, test_x, train_y, folds, f1_scorer)
cross(svc_classifier, test_x,test_y)

Best hyper-parameters: {'C': 10, 'gamma': 0.0001, 'kernel': 'linear'}
F1 = 0.929
Cross-validation predictions:
f1 = 0.467
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        44
           1       0.61      1.00      0.76        70

    accuracy                           0.61       114
   macro avg       0.31      0.50      0.38       114
weighted avg       0.38      0.61      0.47       114



### DecisionTreeClassifier

In [24]:
load_classifier(decision_classifier, decision_param ,train_x, test_x, train_y,folds, f1_scorer)
cross(decision_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': 3, 'max_features': 2, 'min_samples_leaf': 2, 'presort': True}
F1 = 0.904
Cross-validation predictions:
f1 = 0.938
              precision    recall  f1-score   support

           0       0.95      0.89      0.92        44
           1       0.93      0.97      0.95        70

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



### NaiveBayes

In [25]:
load_classifier(naive_classifier, naive_param,train_x, test_x, train_y, folds, f1_scorer)
cross(naive_classifier, test_x,test_y)

Best hyper-parameters: {'var_smoothing': 1e-09}
F1 = 0.911
Cross-validation predictions:
f1 = 0.930
              precision    recall  f1-score   support

           0       0.93      0.89      0.91        44
           1       0.93      0.96      0.94        70

    accuracy                           0.93       114
   macro avg       0.93      0.92      0.93       114
weighted avg       0.93      0.93      0.93       114



### RandomForestCLassifier

In [26]:
load_classifier(forest_classifier, forest_param, train_x, test_x, train_y,folds, f1_scorer)
cross(forest_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': None, 'max_features': 'auto', 'n_estimators': 250}
F1 = 0.938
Cross-validation predictions:
f1 = 0.921
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        44
           1       0.93      0.94      0.94        70

    accuracy                           0.92       114
   macro avg       0.92      0.91      0.92       114
weighted avg       0.92      0.92      0.92       114



### AdaBoostClassifier

In [27]:
load_classifier(boost_classifier, boost_param, train_x, test_x, train_y,folds, f1_scorer)
cross(boost_classifier, test_x,test_y)

Best hyper-parameters: {'algorithm': 'SAMME', 'learning_rate': 1, 'n_estimators': 50}
F1 = 0.947
Cross-validation predictions:
f1 = 0.938
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        44
           1       0.94      0.96      0.95        70

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



### VotingCLassifier

In [28]:
load_classifier(voting_classifier, voting_param, train_x, test_x, train_y, folds, f1_scorer)
cross(voting_classifier, test_x,test_y)

Best hyper-parameters: {'flatten_transform': True, 'voting': 'soft'}
F1 = 0.911
Cross-validation predictions:
f1 = 0.938
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        44
           1       0.94      0.96      0.95        70

    accuracy                           0.94       114
   macro avg       0.94      0.93      0.93       114
weighted avg       0.94      0.94      0.94       114



## Wine

In [29]:
model = dummy.DummyClassifier(strategy='most_frequent')
model.fit (data_w, target_w)
predictions = model.predict(data_w)
accuracy_base = metrics.accuracy_score(target_w, predictions)
print ("Accuracy = {:.3f}".format(accuracy_base))
print(metrics.classification_report(target_w, predictions))
#metrics.plot_confusion_matrix(model, data_d, target_d)

Accuracy = 0.399
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        59
           1       0.40      1.00      0.57        71
           2       0.00      0.00      0.00        48

    accuracy                           0.40       178
   macro avg       0.13      0.33      0.19       178
weighted avg       0.16      0.40      0.23       178



### Find best classifier

In [30]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(data_w, target_w, test_size = 0.2)
folds = 5
f1_scorer = metrics.make_scorer(metrics.f1_score, average='weighted')

### LogisticRegression

In [31]:
load_classifier(logistic_classifier,logistic_param,train_x, test_x, train_y, folds, f1_scorer)
cross(logistic_classifier, test_x,test_y)

Best hyper-parameters: {'C': 10, 'max_iter': 100, 'penalty': 'l1', 'tol': 0.001}
F1 = 0.944
Cross-validation predictions:
f1 = 0.750
              precision    recall  f1-score   support

           0       0.86      0.86      0.86        14
           1       0.77      0.77      0.77        13
           2       0.56      0.56      0.56         9

    accuracy                           0.75        36
   macro avg       0.73      0.73      0.73        36
weighted avg       0.75      0.75      0.75        36



### SVM

In [32]:
load_classifier(svc_classifier, svc_param, train_x, test_x, train_y, folds, f1_scorer)
cross(svc_classifier, test_x,test_y)

Best hyper-parameters: {'C': 1, 'gamma': 0.0001, 'kernel': 'linear'}
F1 = 0.944
Cross-validation predictions:
f1 = 0.320
              precision    recall  f1-score   support

           0       0.40      0.71      0.51        14
           1       0.36      0.31      0.33        13
           2       0.00      0.00      0.00         9

    accuracy                           0.39        36
   macro avg       0.25      0.34      0.28        36
weighted avg       0.29      0.39      0.32        36



### DecisionTreeClassifier

In [33]:
load_classifier(decision_classifier, decision_param ,train_x, test_x, train_y,folds, f1_scorer)
cross(decision_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': 5, 'max_features': None, 'min_samples_leaf': 3, 'presort': True}
F1 = 0.889
Cross-validation predictions:
f1 = 0.747
              precision    recall  f1-score   support

           0       0.80      0.86      0.83        14
           1       0.67      0.62      0.64        13
           2       0.78      0.78      0.78         9

    accuracy                           0.75        36
   macro avg       0.75      0.75      0.75        36
weighted avg       0.75      0.75      0.75        36



### NaiveBayes

In [34]:
load_classifier(naive_classifier, naive_param,train_x, test_x, train_y, folds, f1_scorer)
cross(naive_classifier, test_x,test_y)

Best hyper-parameters: {'var_smoothing': 1e-05}
F1 = 0.916
Cross-validation predictions:
f1 = 0.972
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.93      1.00      0.96        13
           2       1.00      0.89      0.94         9

    accuracy                           0.97        36
   macro avg       0.98      0.96      0.97        36
weighted avg       0.97      0.97      0.97        36



### RandomForestCLassifier

In [35]:
load_classifier(forest_classifier, forest_param, train_x, test_x, train_y,folds, f1_scorer)
cross(forest_classifier, test_x,test_y)

Best hyper-parameters: {'max_depth': None, 'max_features': 'auto', 'n_estimators': 250}
F1 = 1.000
Cross-validation predictions:
f1 = 0.886
              precision    recall  f1-score   support

           0       0.88      1.00      0.93        14
           1       0.91      0.77      0.83        13
           2       0.89      0.89      0.89         9

    accuracy                           0.89        36
   macro avg       0.89      0.89      0.89        36
weighted avg       0.89      0.89      0.89        36



### AdaBoostClassifier

In [36]:
load_classifier(boost_classifier, boost_param, train_x, test_x, train_y,folds, f1_scorer)
cross(boost_classifier, test_x,test_y)

Best hyper-parameters: {'algorithm': 'SAMME.R', 'learning_rate': 0.7, 'n_estimators': 100}
F1 = 0.972
Cross-validation predictions:
f1 = 0.728
              precision    recall  f1-score   support

           0       0.67      0.71      0.69        14
           1       0.64      0.69      0.67        13
           2       1.00      0.78      0.88         9

    accuracy                           0.72        36
   macro avg       0.77      0.73      0.74        36
weighted avg       0.74      0.72      0.73        36



### VotingCLassifier

In [37]:
load_classifier(voting_classifier, voting_param, train_x, test_x, train_y, folds, f1_scorer)
cross(voting_classifier, test_x,test_y)

Best hyper-parameters: {'flatten_transform': True, 'voting': 'hard'}
F1 = 1.000
Cross-validation predictions:
f1 = 0.972
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        14
           1       0.93      1.00      0.96        13
           2       1.00      0.89      0.94         9

    accuracy                           0.97        36
   macro avg       0.98      0.96      0.97        36
weighted avg       0.97      0.97      0.97        36



Digits
Baseline: 0.102
Logistic Regression: 0,969
SVM: 0.989
Decision Tree Classifier: 0.863
Naive Bayes: 0.895
Random Forest Classifier:0.980
Ada Boost Classifier: 0.803
Voting Classifier: 0.964

breast_cancer
Baseline: 0.627
Logistic Regression: 0.938
SVM: 0.929
Decision Tree Classifier: 0.904
Naive Bayes: 0.911
Random Forest Classifier:0.938
Ada Boost Classifier: 0.847
Voting Classifier: 0.911

wine 
breast_cancer
Baseline: 0.399
Logistic Regression: 0.944
SVM: 0.944
Decision Tree Classifier: 0.889
Naive Bayes: 0.916
Random Forest Classifier:1.000
Ada Boost Classifier: 0.972
Voting Classifier: 1.000